# YOLOv4 Training & Inference (Colab) — Darknet (AlexeyAB)
This notebook automates training YOLOv4 on the **JayGala pothole dataset** using AlexeyAB's Darknet in Google Colab (GPU). It also shows inference and the dimension estimation pipeline from the paper.

**Notes:**
- Run this notebook in Google Colab with a GPU runtime (Runtime → Change runtime type → GPU).
- The notebook compiles Darknet in Colab and trains using the provided `yolov4.cfg` adapted for `classes=1`.
- You can also modify steps to use a PyTorch YOLOv4 repo instead, but this notebook uses Darknet for fidelity with the paper.


In [1]:
# 1) Setup: clone darknet (AlexeyAB) and compile with GPU/CUDNN support in Colab
!git clone https://github.com/AlexeyAB/darknet.git /content/darknet
%cd /content/darknet
# Enable GPU and CUDNN in Makefile (Colab has CUDA available). We edit the Makefile to enable GPU, CUDNN, OPENCV
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
# compile (this may take a few minutes)
!make -j$(nproc)
%cd /content


c:\content\darknet


fatal: destination path '/content/darknet' already exists and is not an empty directory.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or batch file.
'sed' is not recognized as an internal or external command,
operable program or bat

c:\content



operable program or batch file.


In [2]:
# 2) Download pre-trained convolutional weights (helps faster convergence)
!wget -O /content/darknet/yolov4.conv.137 https://pjreddie.com/media/files/yolov4.conv.137 || true
!ls -lh /content/darknet/yolov4.conv.137


'wget' is not recognized as an internal or external command,
operable program or batch file.
'true' is not recognized as an internal or external command,
operable program or batch file.


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# 3) Clone JayGala pothole dataset into /content/dataset/jaygala
!git clone https://github.com/jaygala24/pothole-detection.git /content/dataset/jaygala || true
!ls -la /content/dataset/jaygala | head -n 40


fatal: destination path '/content/dataset/jaygala' already exists and is not an empty directory.
'true' is not recognized as an internal or external command,
operable program or batch file.
'ls' is not recognized as an internal or external command,
operable program or batch file.
'ls' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# 4) Prepare darknet dataset structure (data/obj contains images and labels)
import os, shutil, glob
src = '/content/dataset/jaygala'
dst = '/content/darknet/data/obj'
os.makedirs(dst, exist_ok=True)
# The JayGala repo typically contains images in 'images' and labels in 'labels' or similar.
imgs = glob.glob(os.path.join(src, '**', '*.jpg'), recursive=True) + glob.glob(os.path.join(src, '**','*.png'), recursive=True)
labels = glob.glob(os.path.join(src, '**', '*.txt'), recursive=True)
print('Found', len(imgs), 'images and', len(labels), 'txt label files')
# Copy images and labels - careful to only copy matching pairs
copied = 0
for img in imgs:
    base = os.path.splitext(os.path.basename(img))[0]
    # try multiple label name patterns
    label_candidates = [os.path.join(os.path.dirname(img), base+'.txt'), os.path.join(src, base+'.txt')]
    target_img = os.path.join(dst, os.path.basename(img))
    shutil.copy(img, target_img)
    # attempt to copy label
    for l in label_candidates:
        if os.path.exists(l):
            shutil.copy(l, os.path.join(dst, os.path.basename(l)))
            break
    copied += 1
print('Copied', copied, 'images to', dst)
!


Found 0 images and 0 txt label files
Copied 0 images to /content/darknet/data/obj


In [5]:
# 5) Create train.txt (list of image paths), obj.names (class names) and obj.data
import glob
img_paths = glob.glob('/content/darknet/data/obj/*.jpg') + glob.glob('/content/darknet/data/obj/*.png')
train_txt = '/content/darknet/data/train.txt'
with open(train_txt, 'w') as f:
    for p in img_paths:
        f.write(p+'\n')
print('Wrote', len(img_paths), 'to', train_txt)
# write obj.names
with open('/content/darknet/data/obj.names','w') as f:
    f.write('pothole\n')
# write obj.data
with open('/content/darknet/data/obj.data','w') as f:
    f.write('classes = 1\n')
    f.write('train = data/train.txt\n')
    f.write('valid = data/train.txt\n')
    f.write('names = data/obj.names\n')
    f.write('backup = /content/darknet/backup\n')
print('Created obj.data and obj.names')


Wrote 0 to /content/darknet/data/train.txt
Created obj.data and obj.names


In [6]:
import re, os
cfg_src = '/content/pothole_project/yolov4.cfg'
cfg_dst = '/content/darknet/cfg/yolov4_pothole.cfg'
os.makedirs('/content/darknet/cfg', exist_ok=True)
if not os.path.exists(cfg_src):
    print('yolov4.cfg not found. Please upload or copy it to /content/pothole_project/')
    # If running locally, upload/copy manually
else:
    with open(cfg_src,'r') as f:
        cfg = f.read()
    cfg = re.sub(r'max_batches\\s*=\\s*\\d+', 'max_batches = 6000', cfg)
    with open(cfg_dst,'w') as f:
        f.write(cfg)
    print('Wrote patched cfg to', cfg_dst)

yolov4.cfg not found. Please upload or copy it to /content/pothole_project/



In [7]:
# 7) Training command (run this cell to start training). Training will use yolov4.conv.137 as pretrained weights.
get_ipython().system_raw('./darknet/darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov4_pothole.cfg /content/darknet/yolov4.conv.137 -dont_show -map &')
print('Training started in background. Check Darknet output in the notebook logs (tail -f /content/darknet/chart.png or use !tail -f ...)')


Training started in background. Check Darknet output in the notebook logs (tail -f /content/darknet/chart.png or use !tail -f ...)


In [8]:
# 8) After training, run inference on a sample image (replace with your image path)
sample_img = '/content/darknet/data/obj/000001.jpg'  # change if not present
output_img = '/content/darknet/pred.jpg'
# weights path - change to your latest backup weights (e.g., /content/darknet/backup/yolov4_pothole_final.weights)
weights = '/content/darknet/backup/yolov4_pothole_last.weights'
print('Run this once you have weights:', weights)
!./darknet/darknet detector test /content/darknet/data/obj.data /content/darknet/cfg/yolov4_pothole.cfg {weights} {sample_img} -dont_show -ext_output -out /content/darknet/predictions.json || true
from IPython.display import Image, display
if os.path.exists('/content/darknet/pred.jpg'):
    display(Image('/content/darknet/pred.jpg'))
else:
    print('Prediction image not found; check detection command output or replace sample_img with a valid image')


Run this once you have weights: /content/darknet/backup/yolov4_pothole_last.weights
Prediction image not found; check detection command output or replace sample_img with a valid image
Prediction image not found; check detection command output or replace sample_img with a valid image


'.' is not recognized as an internal or external command,
operable program or batch file.
'true' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
# 9) Example: parse darknet JSON output and compute sizes using triangular similarity
import json, os
pred_json = '/content/darknet/predictions.json'
if os.path.exists(pred_json):
    preds = json.load(open(pred_json))
    # predictions are in COCO-like format per image; we'll extract bounding boxes
    for img_pred in preds:
        img_path = img_pred.get('image_id')
        predictions = img_pred.get('objects', [])
        print('Image:', img_path, 'detections:', len(predictions))
        # Example: compute sizes using the dimension_estimation.py functions (we'll copy module into /content)
else:
    print('No predictions.json found. Run the detection command from previous cell producing -out predictions.json')


No predictions.json found. Run the detection command from previous cell producing -out predictions.json


In [12]:
# 10) Copy the dimension_estimation module from the project into /content and show an example usage
import sys, os
sys.path.append('/content')  # Ensure /content is in Python path
!cp -v /content/pothole_project/dimension_estimation.py /content/dimension_estimation.py || true
from dimension_estimation import compute_perceived_focal_length, pixels_to_cm
print('Module imported. Example: compute focal length:')
P = 200  # pixel width measured of known object in image
W = 30.0 # known real width in cm
D = 90.0 # distance in cm (camera height)
F = compute_perceived_focal_length(P, W, D)
print('Perceived focal length F =', F)
print('Convert a bbox width of 120 pixels to cm:', pixels_to_cm(120, F, D))

'cp' is not recognized as an internal or external command,
operable program or batch file.
'true' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'dimension_estimation'

In [ ]:
# If running locally, manually set the image path
img_path = '/content/your_image.jpg'  # Change to your image path

# Run Darknet prediction
cfg_path = '/content/darknet/cfg/yolov4_pothole.cfg'
data_path = '/content/darknet/data/obj.data'
weights_path = '/content/darknet/backup/yolov4_pothole_last.weights'  # Change to your weights
output_img = '/content/darknet/pred.jpg'
!./darknet/darknet detector test {data_path} {cfg_path} {weights_path} {img_path} -dont_show -ext_output -out /content/darknet/predictions.json || true

# Show prediction result
from IPython.display import display, Image as IPyImage
import os
if os.path.exists(output_img):
    display(IPyImage(output_img))
else:
    print('Prediction image not found; check detection command output or uploaded image path.')

Prediction image not found; check detection command output or uploaded image path.


'.' is not recognized as an internal or external command,
operable program or batch file.
'true' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# Fix for ipywidgets FileUpload in Colab/Jupyter
# This cell works in Jupyter, but Colab's FileUpload returns bytes differently.
# If you get an error, use this version for Colab:

import os
from IPython.display import display, Image as IPyImage
import ipywidgets as widgets

file_upload = widgets.FileUpload(accept='.jpg,.png', multiple=False)
predict_btn = widgets.Button(description='Upload & Predict')

# Works in Jupyter (ipywidgets >=7.6)
def predict_image(change=None):
    if not file_upload.value:
        print('No file uploaded.')
        return
    # For Jupyter: file_upload.value is a dict of filename: dict
    # For Colab: file_upload.value is a list of dicts
    try:
        # Try Jupyter style
        fname = list(file_upload.value.keys())[0]
        content = file_upload.value[fname]['content']
    except Exception:
        # Try Colab style
        fname = file_upload.value[0]['metadata']['name']
        content = file_upload.value[0]['content']
    img_path = f"/content/{fname}"
    with open(img_path, "wb") as f:
        f.write(content)
    print(f"Image saved to {img_path}")
    # Run Darknet prediction
    cfg_path = '/content/darknet/cfg/yolov4_pothole.cfg'
    data_path = '/content/darknet/data/obj.data'
    weights_path = '/content/darknet/backup/yolov4_pothole_last.weights'  # Change to your weights
    output_img = '/content/darknet/pred.jpg'
    os.system(f"./darknet/darknet detector test {data_path} {cfg_path} {weights_path} {img_path} -dont_show -ext_output -out /content/darknet/predictions.json")
    if os.path.exists(output_img):
        display(IPyImage(output_img))
    else:
        print('Prediction image not found; check detection command output or uploaded image path.')

predict_btn.on_click(predict_image)
display(file_upload, predict_btn)

ModuleNotFoundError: No module named 'ipywidgets'